In [ ]:
import pickle
import itertools
import open3d as o3d
import numpy as np
import matplotlib.pyplot as plt
import os
import cv2
import pathlib

In [ ]:
## mask.csvように書き換える
class Camera:
    def __init__(self, img_num, f=8000 / 3, cx=1920 / 2, cy=1080 / 2):
        self.img_num = img_num  # カメラ番号（int;コンストラクタ）

        A = np.array([[f, 0, cx], [0, f, cy], [0, 0, 1]])

        self.A = A  # 内部パラメータ(ndarray)後から更新

    def img_load(self, dir_path):
        """_summary_

        Args:
            file_path (_type_): _description_

        TODO: 画像を1チャネルに変える
        """
        img_list = []
        for i in range(8):
            file_path = os.path.join(dir_path, "{}_{}.png".format(self.img_num, i))
            img = cv2.imread(file_path, 1)  # BGRで読み込み
            img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
            img = cv2.flip(img, 1)
            img_list.append(img)
        self.img = img_list  # 画像(ndarray)
        self.img_shape = img.shape

    def contour_extraction(
        self,
        labels=[
            [255, 0, 0],
            [0, 255, 0],
            [0, 0, 255],
            [255, 255, 0],
            [255, 0, 255],
            [0, 255, 255],
            [127, 127, 127],
            [127, 0, 127]
        ],
    ):
        """Extract contours based on their labels (colors)

        Args:
            labels (_type_): _description_

        TODO: 画像を1チャネルに変える
        """

        n_labels = len(labels)
        color_arr = np.array(labels, dtype=np.int16)
        masks = np.ones(
            (self.img[0].shape[0], self.img[0].shape[1], n_labels), dtype=np.uint8
        )

        for i, color in enumerate(color_arr):
            lower = np.clip(color, 0, 255)
            upper = np.clip(color, 0, 255)
            img_mask = cv2.inRange(self.img[i], lower, upper)
            masks[:, :, i] = img_mask

        self.masks = masks # 色ごとのマスク(nd.array)

        contour_list = []

        # 色ごとに輪郭（閉曲線）を抽出
        for i in range(masks.shape[2]):
            contours, hierarchy = cv2.findContours(
                masks[:, :, i], cv2.RETR_TREE, cv2.CHAIN_APPROX_NONE
            )
            contour_list.append(contours)
        self.contour_list = contour_list  # 輪郭のリスト(list,ndarray)

    def para_load(self, file_path):
        self.Rt = np.loadtxt(file_path, delimiter="\t")
        self.P = np.dot(self.A, self.Rt[0:3, 0:4])
        self.cam_world_cood = -np.dot(self.Rt[0:3, 0:3].T, self.Rt[0:3, 3])

def norm(vec):
    s_vec = np.square(vec)
    sume = np.sum(s_vec)
    ss = np.sqrt(sume)
    return vec/ss

class LabelProp:
    
    def __init__(self, val, pcd, labels, img_folder_path, view_mat_folder_path):
        self.num = val
        self.LABELS = labels
        self.PCD = pcd
        self.im_path = img_folder_path
        self.view_path = view_mat_folder_path
        
    def cam_load(self):
        test = Camera(self.num)
        test.img_load(self.im_path)

        test.contour_extraction()
        test.para_load(os.path.join(self.view_path, "{}.csv".format(str(self.num))))
        self.cam = test
    
    def point_cloud_load(self):
        self.pcd = self.PCD
        self.labels = self.LABELS
        u, counts = np.unique(self.labels, return_counts=True)
        self.pt_counts = counts
        
    
    def pc_prop(self):
        # 点群加工
        camera = self.cam.cam_world_cood   
        diameter = np.linalg.norm(
            np.asarray(self.pcd.get_max_bound()) - np.asarray(self.pcd.get_min_bound()))
        #camera = cam_world_cood
        radius = diameter*10000
        
        self.pre_pt = pcd.points
        self.pre_pt = np.concatenate([np.asarray(self.pre_pt), np.ones((np.asarray(self.pre_pt).shape[0],1))], axis=1)

        _, pt_map = self.pcd.hidden_point_removal(camera, radius)
        self.pcd = self.pcd.select_by_index(sorted(pt_map))

        self.TD_points = np.concatenate([np.asarray(self.pcd.points), np.ones((np.asarray(self.pcd.points).shape[0],1))], axis=1)
        self.pre_labels = self.labels
        self.labels = self.labels[np.array(sorted(pt_map))]
        
    def pc_labeling_menber(self):
        self.repro_points, self.labels = pc_labeling(self.TD_points, self.cam.P, self.cam.img_shape, self.labels)
        self.pre_repro_pt, self.pre_labels = pc_labeling(self.pre_pt, self.cam.P, self.cam.img_shape, self.pre_labels)
        
    def labeling(self):
        overrap_list = np.zeros((labels.max().astype(int)+1 , self.cam.masks.shape[2]))
        for j in np.unique(self.labels):
            temp_overrap_list = []
            zero_mask = np.zeros([self.cam.masks.shape[0],self.cam.masks.shape[1]])
            #pre_zero_mask = np.zeros([self.cam.masks.shape[0],self.cam.masks.shape[1]])
            zero_mask[self.repro_points[self.labels==j][:,1], self.repro_points[self.labels==j][:,0]] = 1
            #pre_zero_mask[self.pre_repro_pt[self.pre_labels==j][:,1], self.pre_repro_pt[self.pre_labels==j][:,0]] = 1
            for i in range(self.cam.masks.shape[2]):
                temp_overrap_list.append(np.sum((zero_mask>0) * (self.cam.masks[:,:,i]>0)))
            overrap_list[int(j),:] = np.array(temp_overrap_list)
        all_zero_idx = np.all(overrap_list==0, axis=0)
        label = np.argmax(overrap_list, axis=0).astype("float64")
        self.repro_point_num = np.max(overrap_list, axis=1)
        label[all_zero_idx] = np.nan
        self.output_label = label
    
    def calc_occlusion(self):
        counts = self.pt_counts
        occlusion = self.repro_point_num/counts
        self.occlusion = occlusion

def pc_labeling(TD_points, P_mat, img_shape, labels):
    repro_points = normalization((P_mat@TD_points.T).T)
    repro_points = np.round(repro_points).astype(np.int64)    
    idx = (repro_points[:,0]>0) & (repro_points[:,0]<img_shape[1]) & (repro_points[:,1]>0) & (repro_points[:,1]<img_shape[0])
    repro_points = repro_points[idx]
    labels = labels[idx]
    return repro_points, labels
        
def normalization(vec3):
    return np.array([vec3[:,0]/vec3[:,2], vec3[:,1]/vec3[:,2]]).T

        
def prop_all(num, pcd, labels, img_folder_path, view_mat_folder_path):
    lp = LabelProp(num, pcd, labels, img_folder_path, view_mat_folder_path)
    lp.cam_load()
    lp.point_cloud_load()
    lp.pc_prop()
    lp.pc_labeling_menber()
    lp.labeling()
    lp.calc_occlusion()
    return lp.output_label, lp.occlusion

def rotation_mat(angle):
    Rx = np.array([[1,0,0],
                 [0, np.cos(angle[0]), -np.sin(angle[0])],
                 [0, np.sin(angle[0]), np.cos(angle[0])]])

    Ry = np.array([[np.cos(angle[1]), 0, np.sin(angle[1])],
                 [0,1,0],
                 [-np.sin(angle[1]), 0, np.cos(angle[1])]])

    Rz = np.array([[np.cos(angle[2]), -np.sin(angle[2]), 0],
                 [np.sin(angle[2]), np.cos(angle[2]), 0],
                 [0,0,1]])
    return Rz@Rx@Ry
angle = [np.pi, np.pi, 0] # pcd.rotate()
#angle = [-np.pi/2, np.pi/2, np.pi/2]
R = rotation_mat(angle)
R_mirror = np.array([[-1,0,0],
                    [0,1,0],
                    [0,0,1]])

def clustering(mesh_path):
    mesh = o3d.io.read_triangle_mesh(mesh_path)
    pcd = mesh.sample_points_uniformly(number_of_points=150000)
    pcd.rotate(R,(0,0,0))
    points = np.asarray(pcd.points)
    points = points[:,[0,2,1]]
    pcd.points = o3d.utility.Vector3dVector(points)

    labels = np.array(pcd.cluster_dbscan(eps=0.075, min_points=100))
    
    pcd.scale(0.1,(0,0,0))
    #pcd.points = o3d.utility.Vector3dVector((np.asarray(pcd.points)/10))
    max_label = labels.max()
    colors = plt.get_cmap("tab20")(labels / (max_label if max_label > 0 else 1))
    colors[labels < 0] = 0
    pcd.colors = o3d.utility.Vector3dVector(colors[:, :3])
    #o3d.visualization.draw_geometries([pcd])
    return labels, pcd

In [ ]:
mesh_path = "MESH_PATH"
image_folder = "IMAGE_PATH"
view_mat_path = "VIWE_MAT_PATH"
img_num = 0    # image number

mesh_path = pathlib.Path(mesh_path)
image_folder = pathlib.Path(image_folder)
view_mat_path = pathlib.Path(view_mat_path)

In [ ]:
# demo
mesh_path = pathlib.Path("data/polygon/demo_poly.ply")
image_folder = pathlib.Path("data/image/demo_image/")
view_mat_path = pathlib.Path("data/view_mat/demo_view_mat/")
img_num = 32

In [ ]:
os.makedirs(os.path.join("data", "labels"), exist_ok=True)
image_num_folder = list(image_folder.glob("*"))

labels, pcd = clustering(str(mesh_path))
#labels, pcd.points = labels[np.where(labels>=0)], pcd.points[np.where(labels>=0)]

l = []
o = []
for k in range(img_num): 
    label, occlusion = prop_all(k, pcd, labels, str(image_folder), str(view_mat_path))
    l.append(label)
    o.append(occlusion)

with open(os.path.join("data", "labels", '{}_label.pickle'.format(mesh_path.stem)), 'wb') as f:
    pickle.dump(l, f)
with open(os.path.join("data", "labels", '{}_occlusion.pickle'.format(mesh_path.stem)), 'wb') as f:
    pickle.dump(o, f)